In [1]:
import pandas as pd
import s3fs
from smart_open import open
import boto3
from io import StringIO # python3; python2: BytesIO 
from boto3.s3.transfer import TransferConfig
import metrics
import torch
from transformers import *
import numpy as np
import ast
import time

In [2]:
pd.set_option('display.max_colwidth', -1)

In [4]:
user_tokens = pd.read_csv('s3://recsys-challenge-2020/user_tokens.csv')
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
model = BertModel.from_pretrained('/dev/bert/')

In [7]:
user_tokens.head()

,engaging_user_id,text_ tokens
0,000000E4C259FAD27812910B1DF398D7,"['101\t56898\t137\t38516\t14013\t73477\t16105\t11010\t131\t31558\t10160\t10435\t10105\t82808\t10189\t10309\t62052\t10142\t10957\t10111\t10226\t15243\t10160\t10105\t16700\t100\t108\t56658\t96325\t36535\t35999\t108\t56658\t69184\t89696\t11281\t14120\t131\t120\t120\t188\t119\t11170\t120\t65589\t10280\t10133\t11447\t10112\t11274\t100\t102', '101\t56898\t137\t24090\t19779\t168\t168\t131\t146\t20970\t10472\t13246\t11783\t10114\t38456\t100\t187\t12360\t102']"
1,000006829BEADA9EEA695CF0C334B426,"['101\t2435\t5656\t2594\t8279\t8623\t1925\t64126\t14725\t16867\t2072\t7744\t8137\t6224\t3446\t3189\t1939\t15355\t102', '101\t5954\t3365\t6457\t26922\t22946\t108247\t100\t119\t6909\t1980\t3125\t12290\t6089\t3999\t1980\t4870\t2117\t6563\t4844\t10055\t1942\t3195\t1946\t18226\t15970\t64184\t14750\t10898\t4777\t18280\t12236\t22445\t100\t3197\t3184\t1939\t14803\t99623\t10898\t25204\t8406\t1906\t14803\t53106\t100\t102', '101\t2046\t41368\t47665\t13913\t12230\t47665\t60421\t15970\t102', '101\t8137\t1966\t22162\t35331\t13913\t10055\t119\t1894\t152\t11090\t11369\t108\t17131\t1895\t1910\t26554\t15221\t30884\t10634\t1890\t8613\t2029\t2870\t1998\t7803\t1891\t1894\t1985\t92240\t1895\t14120\t131\t120\t120\t188\t119\t11170\t120\t73845\t10350\t13966\t11447\t49277\t10418\t13034\t10112\t137\t17665\t108247\t16867\t102', '101\t2621\t5618\t2452\t8133\t10497\t50508\t13894\t111756\t119\t108\t127\t1894\t2042\t29004\t1895\t3719\t6457\t1946\t1890\t5086\t7204\t3431\t7972\t10060\t13497\t89304\t12396\t11733\t10061\t1891\t1894\t123\t11274\t35066\t119\t1895\t119\t14120\t131\t120\t120\t188\t119\t11170\t120\t180\t10884\t10162\t15774\t11273\t11127\t13966\t14703\t14703\t102', '101\t60104\t112\t187\t29522\t12174\t10347\t34875\t17695\t17864\t10106\t108\t138\t11355\t10686\t11369\t27424\t13268\t10662\t169\t10751\t19072\t13717\t16898\t10741\t10106\t10105\t141\t72679\t79443\t11259\t10107\t12480\t12360\t18417\t100\t119\t119\t14120\t131\t120\t120\t188\t119\t11170\t120\t149\t12674\t11273\t10929\t104847\t10874\t11403\t10123\t13034\t102', '101\t4476\t4348\t10296\t4388\t10906\t12236\t22445\t10055\t119\t108\t2182\t5478\t123\t102', '101\t95154\t8137\t1966\t5318\t1915\t10055\t102', '101\t62342\t10858\t54439\t19571\t22480\t7831\t8129\t2215\t4671\t7229\t4368\t2226\t3268\t1965\t53106\t119\t14120\t131\t120\t120\t188\t119\t11170\t120\t177\t11779\t10418\t10457\t11274\t10874\t11733\t12674\t12396\t10350\t102', '101\t58955\t10898\t103305\t1901\t16181\t7168\t17984\t15221\t2149\t1980\t7168\t17984\t15221\t4388\t1943\t7168\t15219\t28224\t27849\t22946\t27607\t119\t1904\t27058\t14244\t5769\t1943\t2738\t1917\t30884\t111806\t35412\t74381\t28224\t50508\t1901\t10060\t112\t489\t112\t10061\t119\t119\t108\t29005\t10230\t108\t7457\t2893\t6101\t14120\t131\t120\t120\t188\t119\t11170\t120\t141\t10305\t25743\t13034\t10729\t11527\t10410\t11369\t102', '101\t1910\t26554\t38423\t11259\t12674\t25204\t26554\t5155\t3200\t102', '101\t1861\t10738\t10884\t106\t105469\t7860\t2316\t8133\t3268\t1861\t119\t2048\t92911\t19763\t35969\t10898\t6207\t2146\t1923\t1881\t46094\t4284\t7719\t7177\t3808\t1991\t33499\t41368\t15713\t15970\t111825\t39104\t1892\t1904\t10634\t8374\t1946\t30696\t78043\t10738\t11281\t3437\t2068\t4376\t3186\t6257\t1893\t1980\t8133\t3268\t17984\t14803\t27058\t16923\t1882\t1910\t111798\t67946\t14244\t11127\t10123\t5468\t3775\t1939\t8215\t3388\t2008\t13913\t78354\t21451\t7406\t6800\t1985\t15970\t64184\t14750\t10898\t58719\t11090\t12236\t16838\t53106\t10055\t7860\t2316\t1947\t1894\t123\t120\t126\t10249\t131\t11867\t1895\t14218\t108\t1991\t25071\t15103\t14120\t131\t120\t120\t188\t119\t11170\t120\t194\t15417\t12022\t10138\t69196\t11779\t10116\t12674\t10174\t102', '101\t56898\t137\t16349\t11530\t11274\t12396\t11779\t11273\t168\t45657\t131\t2038\t78841\t73115\t13894\t5106\t7543\t3670\t1985\t11668\t57743\t21761\t11377\t3393\t4337\t1923\t53106\t10055\t5719\t44360\t10898\t11490\t30297\t10155\t12360\t24310\t11312\t26554\t32308\t56910\t22526\t3820\t15219\t15752\t252

In [10]:
val_set.head()

,text_ tokens,engaging_user_id
0,101\t17713\t9405\t119152\t119002\t118823\t9531\t10530\t13767\t9665\t118649\t33323\t23466\t9405\t18623\t27023\t58303\t48345\t119\t119\t119\t9665\t118649\t33323\t23466\t11018\t9405\t119152\t9531\t11489\t9580\t118762\t9519\t119285\t130\t14040\t17655\t9580\t31531\t125\t14040\t18382\t68943\t9576\t16605\t58303\t48345\t119\t119\t119\t137\t49866\t27572\t10112\t11259\t12674\t11403\t108\t80426\t11403\t108\t9560\t22333\t14120\t131\t120\t120\t188\t119\t11170\t120\t171\t10731\t48205\t11259\t12396\t11211\t11281\t10174\t11090\t102,A4B1828B571271073C68DA93D1465484
1,101\t64002\t9247\t23665\t9251\t17342\t23990\t9764\t36553\t100543\t9358\t56789\t9247\t18108\t12310\t9485\t38709\t12178\t28911\t1729\t1729\t1729\t14120\t131\t120\t120\t188\t119\t11170\t120\t191\t19282\t10305\t83148\t13966\t10305\t13966\t11166\t10161\t102,F3A4C8EB81ADF3685E8F693037808A2C
2,101\t56898\t137\t44592\t11090\t13503\t10157\t131\t17144\t112\t187\t12820\t43662\t10271\t10950\t119\t119\t119\t14156\t10114\t13028\t10479\t10124\t10105\t14361\t10139\t10165\t16297\t96509\t13328\t11762\t10551\t136\t119\t108\t10882\t11759\t11273\t10280\t10305\t108\t31091\t34567\t10237\t10731\t10237\t100\t102,8BD011AB59AC04491687749245009C60
3,101\t137\t15595\t21840\t14590\t75268\t57493\t10759\t872\t13874\t32629\t28462\t102,F7BFBA56274FCBBBB76BDD36D0D6495B
4,101\t56898\t137\t13069\t93870\t10284\t11274\t12396\t168\t60801\t11211\t131\t9331\t30842\t10739\t9580\t118762\t9491\t66540\t17594\t100\t119\t9076\t68833\t8843\t19105\t18108\t9425\t12424\t9390\t119086\t9609\t11018\t8863\t100\t119\t9150\t12965\t8996\t26737\t101202\t9137\t11018\t9144\t9551\t23811\t16439\t100\t14120\t131\t120\t120\t188\t119\t11170\t120\t147\t11373\t11259\t10759\t12674\t10157\t11733\t10116\t11259\t10575\t102,10F6E035959B9AC156CED11E0FAED566


In [9]:
column_of_interest = ["engaging_user_id", "text_ tokens"]
val_set = pd.read_csv('s3://recsys-challenge-2020/val_set_reply.csv', encoding="utf-8",
                     usecols= [1, 4])

In [11]:
user_tokens_val_set = \
pd.merge(val_set, user_tokens, how = 'left', left_on = 'engaging_user_id', right_on = 'engaging_user_id', sort=False)

In [59]:
def calculate_average(row1, row2):
    if pd.isna(row1):
        return 0.5
    sum_tensors = torch.zeros([768], dtype=torch.float32)
    # row1 can be nan, as, there are cold users in validation set.
    tweet_token_list = ast.literal_eval(row1)
    for token_list in tweet_token_list:
        token_list_embeddings = model(torch.tensor(list(map(int, token_list.split('\t')))).unsqueeze(0))[0][0][0]
        sum_tensors = sum_tensors + token_list_embeddings
    avg = sum_tensors/len(row1)
    tweet_average_embedding = model(torch.tensor(list(map(int, row2.split('\t')))).unsqueeze(0))[0][0][0]
    score = torch.dot(avg, tweet_average_embedding)
    return score.detach().numpy().item(0)

In [21]:
user_val_set_reply_score = pd.DataFrame()

In [24]:
user_tokens_val_set.columns = [c.replace(' ', '_') for c in user_tokens_val_set.columns]

In [25]:
user_tokens_val_set.columns

Index(['text__tokens_x', 'engaging_user_id', 'text__tokens_y'], dtype='object')

In [ ]:
user_val_set_reply_score['reply_score'] = user_tokens_val_set.apply \
(lambda z: calculate_average(z.text__tokens_y, z.text__tokens_x), axis = 1)

In [62]:
user_val_set_reply_score.to_csv('s3://recsyschallenge2020/user_val_set_reply_score', index = False)

""


In [ ]:
user_val_set_reply_score

In [ ]:
df['col_3'] = df.apply(lambda x: f(x.col_1, x.col_2), axis=1)

In [ ]:
user_tokens.to_csv('s3://recsyschallenge2020/user_tokens_embeddins.csv', index = False)

In [ ]:
user_tokens.head()

In [ ]:
len(user_tokens)

In [ ]:
input_ids = torch.tensor([101, 108, 31278, 90939, 70325, 196, 199, 71436, 10107, 29190, 10107, 106, 16680, 68314, 10153, 17015, 15934, 10104, 108, 10233, 12396, 14945, 10107, 10858, 11405, 13600, 13597, 169, 57343, 64482, 119, 119, 119, 100, 11741, 16381, 10109, 68830, 10110, 20886, 108, 10233, 11127, 21768, 100, 14120, 131, 120, 120, 188, 119, 11170, 120, 12132, 10884, 10157, 11490, 12022, 10113, 10731, 10729, 11565, 14120, 131, 120, 120, 188, 119, 11170, 120, 162, 11211, 11703, 12022, 11211, 10240, 44466, 100886, 102]).unsqueeze(0)  # Batch size 1
outputs = model(input_ids)
last_hidden_states = outputs[0][0][0]  # The last hidden-state is the first element of the output tuple

In [ ]:
model(torch.tensor(list(map(int, ast.literal_eval(user_tokens.loc[0]['text_ tokens']).split('\t')))).unsqueeze(0))[0][0][0]

In [ ]:
it = 0

start = time.time()

# your code

# end
sum_tensors = torch.zeros([768], dtype=torch.float32)
for index, row in user_tokens.iterrows():
    #print(len(row))
    tweet_token_list = ast.literal_eval(row['text_ tokens'])
    for token_list in tweet_token_list:
        token_list_embeddings = model(torch.tensor(list(map(int, token_list.split('\t')))).unsqueeze(0))[0][0][0]
        sum_tensors = sum_tensors + token_list_embeddings
    if it %100 == 0:
        print(f'Time: {time.time() - start}')
        print(it)
    it = it + 1
    print(it)
    avg = sum_tensors/len(row)

In [ ]:
sum_tensors = torch.zeros([768], dtype=torch.float32)

In [ ]:
sum_tensors

In [ ]:
import dask.dataframe as ddf

In [ ]:
df_dask = ddf.from_pandas(user_tokens, npartitions=32)   # where the number of partitions is the number of cores you want to use

In [ ]:
df_dask.apply(lambda x: calculate_average(x), axis  = 1).compute(scheduler='multiprocessing')